In [ ]:
pip show azure-ai-ml


In [ ]:

# enter details of your AML workspace
subscription_id = "YOUR-SUBSCRIPTION-ID"
resource_group = "rg-churn-pred-proj"
workspace = "churn-pred-proj"

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace